# Preprocess train data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from xgboost import XGBRegressor

from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import mean_squared_error

from joblib import dump
from joblib import load

In [2]:
df = pd.read_csv("../data/Mastercard_stock_history.csv")
df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2006-05-25,3.748967,4.283869,3.739664,4.279217,395343000,0.0,0.0
1,2006-05-26,4.307126,4.348058,4.103398,4.179680,103044000,0.0,0.0
2,2006-05-30,4.183400,4.184330,3.986184,4.093164,49898000,0.0,0.0
3,2006-05-31,4.125723,4.219679,4.125723,4.180608,30002000,0.0,0.0
4,2006-06-01,4.179678,4.474572,4.176887,4.419686,62344000,0.0,0.0


Once again, we try to forecast the close price using previous date features, as well as generating new features for our model to predict the close price

In [3]:
# convert Date into datetime, and set the date as the index of the dataframe
df['Date'] = pd.to_datetime(df['Date'])
# set date as the index
df.set_index('Date', inplace=True)

df = df.asfreq("D")
df = df.interpolate() # interpolate data to remove NaN values

## Create new features for the model

We will create features for:

1. month
2. day of the week
3. moving average
4. moving standard deviation
5. previous days' lagged close prices
6. open price
7. quarter of the year
8. Relative Strength Indicator

In [4]:
df['month'] = df.index.month
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,month
Date,,,,,,,,
2006-05-25,3.748967,4.283869,3.739664,4.279217,395343000.0,0.0,0.0,5
2006-05-26,4.307126,4.348058,4.103398,4.179680,103044000.0,0.0,0.0,5
2006-05-27,4.276195,4.307126,4.074094,4.158051,89757500.0,0.0,0.0,5
2006-05-28,4.245263,4.266194,4.044791,4.136422,76471000.0,0.0,0.0,5
2006-05-29,4.214332,4.225262,4.015487,4.114793,63184500.0,0.0,0.0,5


In [5]:
df['day'] = df.index.day_of_week
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,month,day
Date,,,,,,,,,
2006-05-25,3.748967,4.283869,3.739664,4.279217,395343000.0,0.0,0.0,5,3
2006-05-26,4.307126,4.348058,4.103398,4.179680,103044000.0,0.0,0.0,5,4
2006-05-27,4.276195,4.307126,4.074094,4.158051,89757500.0,0.0,0.0,5,5
2006-05-28,4.245263,4.266194,4.044791,4.136422,76471000.0,0.0,0.0,5,6
2006-05-29,4.214332,4.225262,4.015487,4.114793,63184500.0,0.0,0.0,5,0


In [6]:
df['quarter'] = df.index.quarter
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,month,day,quarter
Date,,,,,,,,,,
2006-05-25,3.748967,4.283869,3.739664,4.279217,395343000.0,0.0,0.0,5,3,2
2006-05-26,4.307126,4.348058,4.103398,4.179680,103044000.0,0.0,0.0,5,4,2
2006-05-27,4.276195,4.307126,4.074094,4.158051,89757500.0,0.0,0.0,5,5,2
2006-05-28,4.245263,4.266194,4.044791,4.136422,76471000.0,0.0,0.0,5,6,2
2006-05-29,4.214332,4.225262,4.015487,4.114793,63184500.0,0.0,0.0,5,0,2


In [7]:
df['lag_1'] = df['Close'].shift(1)
df['lag_2'] = df['Close'].shift(2)

df = df.dropna()
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,month,day,quarter,lag_1,lag_2
Date,,,,,,,,,,,,
2006-05-27,4.276195,4.307126,4.074094,4.158051,89757500.0,0.0,0.0,5,5,2,4.179680,4.279217
2006-05-28,4.245263,4.266194,4.044791,4.136422,76471000.0,0.0,0.0,5,6,2,4.158051,4.179680
2006-05-29,4.214332,4.225262,4.015487,4.114793,63184500.0,0.0,0.0,5,0,2,4.136422,4.158051
2006-05-30,4.183400,4.184330,3.986184,4.093164,49898000.0,0.0,0.0,5,1,2,4.114793,4.136422
2006-05-31,4.125723,4.219679,4.125723,4.180608,30002000.0,0.0,0.0,5,2,2,4.093164,4.114793


In [8]:
# create RSI indicator
df['change'] = df['Close'].diff()
df['change'] = df['change'].shift(1) # lag the change column by 1 time step

df['gain'] = df['change'].apply(lambda x: x if x > 0 else 0)
df['loss'] = df['change'].apply(lambda x: -x if x < 0 else 0)

df['avg_gain'] = df['gain'].rolling(window = 30, min_periods = 1).mean()
df['avg_loss'] = df['loss'].rolling(window = 30, min_periods = 1).mean()

df['RSI'] = 100 - (100 / (1 + (df["avg_gain"] / df["avg_loss"])))
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,month,day,quarter,lag_1,lag_2,change,gain,loss,avg_gain,avg_loss,RSI
Date,,,,,,,,,,,,,,,,,,
2006-05-27,4.276195,4.307126,4.074094,4.158051,89757500.0,0.0,0.0,5,5,2,4.179680,4.279217,NaN,0.0,0.000000,0.0,0.000000,NaN
2006-05-28,4.245263,4.266194,4.044791,4.136422,76471000.0,0.0,0.0,5,6,2,4.158051,4.179680,NaN,0.0,0.000000,0.0,0.000000,NaN
2006-05-29,4.214332,4.225262,4.015487,4.114793,63184500.0,0.0,0.0,5,0,2,4.136422,4.158051,-0.021629,0.0,0.021629,0.0,0.007210,0.0
2006-05-30,4.183400,4.184330,3.986184,4.093164,49898000.0,0.0,0.0,5,1,2,4.114793,4.136422,-0.021629,0.0,0.021629,0.0,0.010814,0.0
2006-05-31,4.125723,4.219679,4.125723,4.180608,30002000.0,0.0,0.0,5,2,2,4.093164,4.114793,-0.021629,0.0,0.021629,0.0,0.012977,0.0


In [9]:
# add ARBR with n = 6, non-lagged
def compute_arbr(df: pd.DataFrame, n = 6, in_place=False, include_lagged=True):
  if not in_place:
    df = df.copy()

  sum_high_open_diff = (df["High"] - df["Open"]).rolling(window=n, min_periods=1).sum()
  sum_open_low_diff = (df["Open"] - df["Low"]).rolling(window=n, min_periods=1).sum()
  df['AR'] = 100 * (sum_high_open_diff / sum_open_low_diff)

  lag_1_close = df["Close"].shift(1)
  max_high_close_diff = (df['High'] - lag_1_close).apply(lambda v: max(0, v))
  max_close_low_diff = (lag_1_close - df['Low']).apply(lambda v: max(0, v))
  df['BR'] = 100 * (max_high_close_diff.rolling(window=n, min_periods=1).sum()
                    / max_close_low_diff.rolling(window=n, min_periods=1).sum())
  if include_lagged:
    df['AR-LAG1'] = df['AR'].shift(1)
    df['BR-LAG1'] = df['BR'].shift(1)

  return df

# add MACD columns (MACD, DEA, MACD-DEA), non-lagged
def compute_macd(df: pd.DataFrame, in_place=False, include_lagged=True):
  ema_12 = df["Close"].ewm(span=12, adjust=False).mean()
  ema_26 = df["Close"].ewm(span=26, adjust=False).mean()
  macd= ema_26 - ema_12
  dea = macd.ewm(span=9, adjust=False).mean()
  df_cp = df

  if not in_place:
    df_cp = df.copy()

  df_cp['DEA'] = dea
  df_cp['MACD'] = macd
  df_cp['MACD-DEA'] = macd - dea
  if include_lagged:
    df_cp['DEA-LAG1'] = df_cp['DEA'].shift(1)
    df_cp['MACD-LAG1'] = df_cp['MACD'].shift(1)
    df_cp['MACD-DEA-LAG1'] = df_cp['MACD-DEA'].shift(1)

  return df_cp 

df_arbr = compute_arbr(df)
df_macd = compute_macd(df_arbr)

df_macd = df_macd.dropna()

# robust scaling on these features: rmse = 7.92
df = df_macd
df_macd[:7]

,Open,High,Low,Close,Volume,Dividends,Stock Splits,month,day,quarter,...,AR,BR,AR-LAG1,BR-LAG1,DEA,MACD,MACD-DEA,DEA-LAG1,MACD-LAG1,MACD-DEA-LAG1
Date,,,,,,,,,,,,,,,,,,,,,
2006-05-29,4.214332,4.225262,4.015487,4.114793,6.318450e+07,0.0,0.0,5,0,2,...,10.440726,84.110692,12.882599,95.481925,0.001233,0.004783,0.003550,0.000345,0.001725,0.001380
2006-05-30,4.183400,4.184330,3.986184,4.093164,4.989800e+07,0.0,0.0,5,1,2,...,7.978942,73.461091,10.440726,84.110692,0.002756,0.008849,0.006093,0.001233,0.004783,0.003550
2006-05-31,4.125723,4.219679,4.125723,4.180608,3.000200e+07,0.0,0.0,5,2,2,...,19.743606,108.332421,7.978942,73.461091,0.003197,0.004959,0.001762,0.002756,0.008849,0.006093
2006-06-01,4.179678,4.474572,4.176887,4.419686,6.234400e+07,0.0,0.0,6,3,2,...,56.471040,187.435339,19.743606,108.332421,-0.000886,-0.017218,-0.016331,0.003197,0.004959,0.001762
2006-06-02,4.511782,4.530387,4.352707,4.371312,3.725300e+07,0.0,0.0,6,4,2,...,58.049489,184.011592,56.471040,187.435339,-0.006817,-0.030537,-0.023721,-0.000886,-0.017218,-0.016331
2006-06-03,4.466820,4.547443,4.359219,4.438292,3.723133e+07,0.0,0.0,6,5,2,...,75.119224,260.484081,58.049489,184.011592,-0.014647,-0.045968,-0.031321,-0.006817,-0.030537,-0.023721
2006-06-04,4.421858,4.564498,4.365731,4.505271,3.720967e+07,0.0,0.0,6,6,2,...,120.818332,318.017817,75.119224,260.484081,-0.024293,-0.062877,-0.038584,-0.014647,-0.045968,-0.031321


In [10]:
# create lag-1 30 day rolling mean and standard deviations
df['MA'] = df["lag_1"].rolling(30).mean()
df['M_STD'] = df["lag_1"].rolling(30).std()

df = df.dropna() # remove NaNs after rolling functions have been applied

In [11]:
columns_to_keep = ["Open", "Volume", "Close", "month", "day", "quarter", "lag_1", "lag_2", "MA", "M_STD"] # ignore RSI, since it makes predictions worse
df1 = df[columns_to_keep]
df1.head()

,Open,Volume,Close,month,day,quarter,lag_1,lag_2,MA,M_STD
Date,,,,,,,,,,
2006-06-27,4.418758,9.826000e+06,4.401083,6,1,2,4.418758,4.401703,4.312141,0.141078
2006-06-28,4.358291,8.756000e+06,4.368524,6,2,2,4.401083,4.418758,4.320963,0.137951
2006-06-29,4.367592,1.447000e+07,4.465270,6,3,2,4.368524,4.401083,4.329421,0.132547
2006-06-30,4.432710,2.596400e+07,4.465270,6,4,2,4.465270,4.368524,4.341824,0.126970
2006-07-01,4.440462,1.965233e+07,4.443563,7,5,3,4.465270,4.465270,4.351313,0.125130


## Standardize data

In [12]:
# scaler = StandardScaler()
# robust_scaler = RobustScaler()
# cols_to_standardize = ['Open', "Volume", "lag_1", "lag_2", "MA", "M_STD"]
# df_scaled = df1.copy()
# df_scaled[cols_to_standardize] = scaler.fit_transform(df1[cols_to_standardize])
# # cols_to_scale = ['AR', 'BR', "RSI"]
# # df1[cols_to_scale] = scaler.fit_transform(df1[cols_to_scale])
# df_scaled.head()

In [13]:
df1.to_csv("../data/processed/mastercard_stock_history_processed.csv", index = True, index_label = "Date")